# BERT with LoRA: Quora Paraphrase Detection

This notebook demonstrates how to use BERT with LoRA for paraphrase detection using the Quora Question Pairs dataset.


In [ ]:
# Install required packages (uncomment if running in Colab)
!pip install torch transformers datasets peft

In [ ]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType

## 1. Load and preprocess the Quora Question Pairs dataset

In [ ]:
# Load dataset
dataset = load_dataset('quora')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(example):
    return tokenizer(
        example['questions']['text'][0],
        example['questions']['text'][1],
        truncation=True,
        padding='max_length',
        max_length=128
    )

encoded = dataset.map(preprocess, batched=False)
encoded = encoded.rename_column('is_duplicate', 'labels')
encoded = encoded.remove_columns(['questions', 'id'])

## 2. Model Setup: BERT + LoRA

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

## 3. Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_paraphrase',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs_paraphrase',
    load_best_model_at_end=True
)

## 4. Training

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded['train'],
    eval_dataset=encoded['validation'],
    tokenizer=tokenizer,
)
trainer.train()

## 5. Evaluation

In [ ]:
results = trainer.evaluate()
print(results)

## 6. Inference Example

In [ ]:
def predict_paraphrase(q1, q2):
    inputs = tokenizer(q1, q2, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    return 'Paraphrase' if pred == 1 else 'Not paraphrase', probs.detach().numpy()

# Example
q1 = 'How can I be a good programmer?'
q2 = 'What should I do to become a better programmer?'
result, probabilities = predict_paraphrase(q1, q2)
print(f'Result: {result}, Probabilities: {probabilities}')